<a href="https://colab.research.google.com/github/Predic-learning/Buiding-Energy/blob/main/Floor_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error, classification_report
data = pd.read_csv('/content/drive/MyDrive/weather/Fl-5.csv')
#data['Tools'].replace({'AIR':0, 'PC':1 , 'lamp':2}, inplace=True)
data.head()

,Class,Tools,Room,Number,BTU,Power,amount,PowerHr,time,SumPower
0,5,AIR,501,1872,38000,3.3,1,3.3,39,128.7
1,5,AIR,502,1873,38000,3.3,1,3.3,39,128.7
2,5,AIR,503,1874,38000,3.3,1,3.3,39,128.7
3,5,AIR,502,1869,38000,3.3,1,3.3,39,128.7
4,5,AIR,503,1870,38000,3.3,1,3.3,39,128.7


In [2]:
num_points = 368
max_error_percent = 20
randomP = [random.uniform(-3.3, 3.3) for _ in range(num_points)]
errorX = [valueX * (max_error_percent / 100) * random.choice([-1, 1]) for valueX in randomP]
dfX = pd.DataFrame({'random_dataX': errorX})
dfX['absolute_random_dataX'] = dfX['random_dataX'].abs()
random_Power = data['PowerHr'] - dfX['absolute_random_dataX']
data = data.assign(random_Power = random_Power)

randomS = [random.uniform(-128.7, 128.7) for _ in range(num_points)]
error = [value * (max_error_percent / 100) * random.choice([-1, 1]) for value in randomS]
df = pd.DataFrame({'random_data': error})
df['absolute_random_data'] = df['random_data'].abs()
random_SumPower = data['SumPower'] - df['absolute_random_data']
data = data.assign(random_SumPower = random_SumPower)
data

,Class,Tools,Room,Number,BTU,Power,amount,PowerHr,time,SumPower,random_Power,random_SumPower
0,5,AIR,501,1872,38000,3.30,1,3.300,39,128.70,2.774877,113.350828
1,5,AIR,502,1873,38000,3.30,1,3.300,39,128.70,3.284894,111.297888
2,5,AIR,503,1874,38000,3.30,1,3.300,39,128.70,2.918298,111.448833
3,5,AIR,502,1869,38000,3.30,1,3.300,39,128.70,2.922484,106.236325
4,5,AIR,503,1870,38000,3.30,1,3.300,39,128.70,2.898106,115.949950
5,5,AIR,504,1871,38000,3.30,1,3.300,39,128.70,3.037873,121.240012
6,5,AIR,503,1866,38000,3.30,1,3.300,36,118.80,3.006922,110.698633
7,5,AIR,504,1867,38000,3.30,1,3.300,36,118.80,3.017147,95.985878
8,5,AIR,505,1868,38000,3.30,1,3.300,36,118.80,2.923858,116.156098
9,5,AIR,504,1863,38000,3.30,1,3.300,30,99.00,3.091981,96.028948


In [3]:
#X = data[['PowerHr','time']].values
#y = data['SumPower'].values.reshape(-1, 1)
X = data[['time', 'random_Power']].values
y = data[['random_SumPower']].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Create a k-Nearest Neighbors (k-NN) regressor
k = 3  # You can choose the value of k
knn_regressor = KNeighborsRegressor(n_neighbors=k)

# Fit the regressor on the training data
knn_regressor.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_regressor.predict(X_test)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse:.2f}")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

r2 = r2_score(y_test, y_pred)
print(f"Accuracy: {r2*100:.2f} %")

MAE: 10.34
MSE: 273.67
RMSE: 16.54
Accuracy: 79.41 %


In [5]:
T= 1
Data = [[data.loc[T, 'time'], data.loc[T, 'random_Power']]]
y_predict = knn_regressor.predict(Data)
y_predict

array([[112.97505641]])